## References:
- Code source - https://github.com/aivclab/detr/tree/8830cacdc981924169546a0e59d94b6c94fd775d
- Install packages - https://stackoverflow.com/questions/66588715/runtimeerror-cudnn-error-cudnn-status-not-initialized-using-pytorch

## Mount google drive

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

## Install packages

In [2]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

1.9.0.post2 False None


## Params

In [5]:
code_dir = "detr/detr" # folder containing source code for DeTR model
outDir = "detr/weights/exp1" # training outputs will be saved in this folder


"""  Training  """

epochs = 50
batch_size = 4


"""  Model loading  """

pretrained = True # use pre-trained model
pretrained_url = 'https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth'
pretrained_download_dir = 'weights/pretrained/detr-r50_no-class-head.pth' # dir to store pre-trained model file


"""  Dataset  """

dataset_file = "coco_custom"
dataDir= "datasets/coco_dataset" # coco dataset folder
num_classes = 2 # actual number of classes + 1 (for no class)
pretrained_file_dir = pretrained_download_dir if pretrained else ""

In [3]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ai-model-training/code/Users/imaduako/detr/detr


## Change dir

In [4]:
#%cd {code_dir}

[Errno 2] No such file or directory: 'detr/detr'
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ai-model-training/code/Users/imaduako/detr/detr


## Load pre-trained model

This greatly depends on the size of a dataset. Smaller datasets rely more on finetuning. 

In [6]:
if pretrained:
    # Get pretrained weights
    checkpoint = torch.hub.load_state_dict_from_url(
                url = pretrained_url,
                map_location = 'cpu',
                check_hash = True)

    # Remove class weights
    del checkpoint["model"]["class_embed.weight"]
    del checkpoint["model"]["class_embed.bias"]

    # SaveOGH
    torch.save(checkpoint,
               pretrained_download_dir)

In [1]:
pip install pycocotools

Processing /home/azureuser/.cache/pip/wheels/3e/08/ac/58126fe59992032701437336493f6132e1b72381a62d00b595/pycocotools-2.0.6-cp38-cp38-linux_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


## Training

We use the main.py script to run our training

In [7]:
!python main.py \
  --dataset_file {dataset_file} \
  --coco_path {dataDir} \
  --output_dir {outDir} \
  --resume {pretrained_file_dir} \
  --num_classes {num_classes} \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs {epochs} \
  --batch_size {batch_size}

Not using distributed mode
git:
  sha: 8830cacdc981924169546a0e59d94b6c94fd775d, status: has uncommited changes, branch: master

Namespace(aux_loss=True, backbone='resnet50', batch_size=4, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path='datasets/coco_dataset', dataset_file='coco_custom', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=50, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=1e-05, lr_backbone=1e-06, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_classes=2, num_queries=100, num_workers=2, output_dir='detr/weights/exp1', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='weights/pretrained/detr-r50_no-class-head.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
Building a DETR model with 2 classes
number of p

In [1]:
import numpy as np
print(np.__version__)

1.19.5


In [ ]:
numpy 1.11.0

## Results

Quick and easy overview of the training results

In [ ]:
from util.plot_utils import plot_logs
from pathlib import Path

In [ ]:
log_directory = [Path(outDir)]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
# Visualize 
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)   

## Prediction on a single image

In [ ]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

### Params

In [ ]:
conf_threshold = 0.5

# COCO classes
CLASSES = ['N/A', 'school']

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125]]

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

### Util functions

In [ ]:
# util functions for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes):
    plt.figgure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

### Load trained model

In [ ]:
trained_checkpoint_dir = os.path.join(outDir, "checkpoint.pth")

# load checkpoint
checkpoint = torch.load(trained_checkpoint_dir, map_location='cpu')

# load model architecture
model = torch.hub.load('facebookresearch/detr', 
                       'detr_resnet50', 
                       pretrained=False, 
                       num_classes=2)
# load weights
model.load_state_dict(checkpoint['model'], strict=False)

### Read image and preprocess

In [ ]:
# sample test image
test_image_dir = 'test_images/gov_137.jpg'

# read image
im = Image.open(test_image_dir)

# mean-std normalize the input image (batch-size: 1)
img = transform(im).unsqueeze(0)

In [ ]:
# propagate through the model
outputs = model(img)

### Post-process prediction results and visualize

In [ ]:
# keep only predictions with 0.7+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
keep = probas.max(-1).values > conf_threshold

# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

plot_results(im, probas[keep], bboxes_scaled)